In [1]:
import json
from typing import Optional

from app.core.plan.evaluator import evaulate_answer
from app.llm.interface import LLMInterface
from app.config.settings import EVALUATION_LLM_PROVIDER, EVALUATION_LLM_MODEL

from notebooks.plan_chat_optimizer import get_task_answer, update_plan, execute_task_using_new_plan
from notebooks.tasks import get_evaluation_pending_tasks, record_evaluation

eval_llm = LLMInterface(EVALUATION_LLM_PROVIDER, EVALUATION_LLM_MODEL)

def optimize_plan(task_id:str, branch_name:Optional[str]="main", max_iteration=3):
    current_branch_name = branch_name
    error_message = None
    iteration_round = 0

    while True:
        iteration_round += 1
        print(f"Start to evaluate plan for task(id={task_id},branch={current_branch_name})")
        detail = get_task_answer(task_id, current_branch_name)

        if detail is not None:
            goal = detail.get("goal")
            final_answer = detail.get("final_answer")
            plan = detail.get("plan")
            metadata = detail.get("metadata")

            if plan is None:
                record_evaluation(task_id, "REJECTED", "No plan found")
                return

            eval_res = evaulate_answer(eval_llm, goal, metadata, final_answer, plan)
            eval_status = "APPROVED" if eval_res.get("accept", False) else "WAITING_FOR_EVALUATION"
            eval_reason = json.dumps(eval_res, indent=2) 

            record_evaluation(task_id, eval_status, eval_reason)

            if eval_res.get("accept", False) is True:
                print(f"Goal Pass! {goal}, evaluation result:{eval_reason}")
                return

            print(f"Goal Not Pass! {goal}, the evaluation result:{eval_reason}")

            if iteration_round >= max_iteration:
                break

            revised_plan = update_plan(goal, metadata, eval_reason, plan)
            print("revised plan:", revised_plan)
            reasoning = revised_plan.get("reasoning", None)
            revised_plan = revised_plan.get("plan", None)

            try:
                updated_result = execute_task_using_new_plan(task_id, revised_plan, reasoning)
                print(f"Revised plan execution result {updated_result}")
            except Exception as e:
                error_message = f"Failed to execute task using new plan {e}"
                break
            
            current_branch_name = updated_result.get("branch_name", None)
            current_final_answer = updated_result.get("final_answer", None)
            if current_branch_name is None or current_final_answer is None:
                error_message = "Failed to execut task using new plan, get empty answer"
                break
    
    if error_message is None:
        error_message = "Still failed after two evaluations round."
    print(f"Failed to evaluate plan for task(id={task_id}): {error_message}")


2025-02-18 19:38:04,193 - apscheduler.scheduler - INFO - Adding job tentatively -- it will be properly scheduled when the scheduler starts
2025-02-18 19:38:04,194 - apscheduler.scheduler - INFO - Added job "SimpleCache.refresh_cache" to job store "default"
2025-02-18 19:38:04,195 - apscheduler.scheduler - INFO - Scheduler started
2025-02-18 19:38:04,195 - app.core.task.simple_cache - INFO - Started cache refresh scheduler to run every 24 hours.
2025-02-18 19:38:04,195 - app.core.task.simple_cache - INFO - Starting cache refresh...
2025-02-18 19:38:14,195 - apscheduler.executors.default - INFO - Running job "SimpleCache.refresh_cache (trigger: interval[1 day, 0:00:00], next run at: 2025-02-18 19:38:14 +08)" (scheduled at 2025-02-18 19:38:14.192442+08:00)
2025-02-18 19:38:14,197 - app.core.task.simple_cache - INFO - Starting cache refresh...
2025-02-18 19:38:23,368 - app.core.task.simple_cache - INFO - Cache refresh completed successfully.


In [2]:
from datetime import datetime, timedelta

end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=2)

pending_tasks = get_evaluation_pending_tasks(
    start_time=start_time
)
pending_tasks

2025-02-18 19:38:28,360 - app.core.task.manager - INFO - Retrieved 18 tasks with evaluation statuses [<EvaluationStatus.NOT_EVALUATED: 'NOT_EVALUATED'>] between 2025-02-18 09:38:27.071674 and 2025-02-18 11:38:27.071725.


[{'id': '2dc87c69-fd03-4c62-93d8-97a3a368e0d8',
  'goal': 'What are the specific advantages and challenges of deploying the PD component in microservices mode in TiDB?',
  'status': 'failed',
  'created_at': datetime.datetime(2025, 2, 18, 11, 5, 57),
  'updated_at': datetime.datetime(2025, 2, 18, 11, 6, 23),
  'logs': 'Execution was interrupted by the client.',
  'best_plan': None,
  'metadata': {'label_path': [{'label': 'Complex Task Planning'},
    {'label': 'Research & Analysis'},
    {'label': 'Technical Design'}],
   'response_format': {'Background': "TiDB's PD component and its role in the architecture, including its functions such as metadata management, scheduling, and high availability. The microservices mode for PD was introduced experimentally in version 8.0.0 to reduce module interference and support larger workloads, as discussed in the previous conversation. Annotations: User is seeking detailed insights into the benefits and potential difficulties associated with deployi

In [3]:
for task in pending_tasks:
    task_id = task["id"]
    optimize_plan(task_id, "main", max_iteration=1)

Start to evaluate plan for task(id=2dc87c69-fd03-4c62-93d8-97a3a368e0d8,branch=main)


2025-02-18 19:38:33,560 - app.core.task.simple_cache - INFO - Cache refresh completed successfully.
2025-02-18 19:38:33,561 - apscheduler.executors.default - INFO - Job "SimpleCache.refresh_cache (trigger: interval[1 day, 0:00:00], next run at: 2025-02-19 19:38:14 +08)" executed successfully


2025-02-18 19:38:35,259 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Goal Not Pass! What are the specific advantages and challenges of deploying the PD component in microservices mode in TiDB?, the evaluation result:{
  "accept": false,
  "plan_adjustment_suggestion": "The Plan lacks dual retrieval, which could enhance the quality of the information gathered. It should incorporate both retrieve_knowledge_graph and vector_search tools to ensure comprehensive data collection. Additionally, the Plan should include immediate processing of combined results through the LLM generation tool to extract key insights and present a coherent narrative. The absence of a Final Answer suggests a possible execution error or incomplete data processing, which needs to be addressed.",
  "goal_classification": "Direct Problem Resolution"
}
Failed to evaluate plan for task(id=2dc87c69-fd03-4c62-93d8-97a3a368e0d8): Still failed after two evaluations round.
Start to evaluate plan for task(id=55ac031c-68a6-429d-8976-a16ca9f3512b,branch=main)


2025-02-18 19:38:40,444 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Goal Pass! What is the maximum number of nodes that a TiDB cluster can have in a real-world scenario?, evaluation result:{
  "accept": true,
  "plan_adjustment_suggestion": "The answer effectively resolves the user's goal by providing detailed information on the scalability limits of TiDB, TiKV, and PD nodes. However, the Plan could be improved by ensuring that the retrieval process includes a step to verify the most current and comprehensive data from both the knowledge graph and vector search. Additionally, the Plan should ensure that the retrieval tools are used in a complementary manner to cover any potential gaps in information. This could involve a more explicit integration of the results from both retrieval methods before generating the final answer.",
  "goal_classification": "Direct Problem Resolution"
}
Start to evaluate plan for task(id=d86ef57b-946a-4d69-b3b1-d9e04beb0bd2,branch=main)


2025-02-18 19:39:57,578 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Goal Pass! How to evaluate the impact of future expansion plans on the choice between TiDB and Citus? What specific measures do TiDB and Citus have for handling data security and privacy? How is Citus's sharding capability specifically implemented in multi-tenant application scenarios? What are some successful cases of TiDB's HTAP capabilities in practical applications? In scenarios involving geographically distributed data, how does Citus ensure data consistency and access speed?, evaluation result:{
  "accept": true,
  "plan_adjustment_suggestion": "The Final Answer effectively addresses the user's Goal by providing a comprehensive comparison of TiDB and Citus across various dimensions such as scalability, security, sharding, HTAP capabilities, and geographic distribution. However, the Plan could be improved by ensuring that each retrieval step is immediately followed by processing through the LLM generation tool to extract key insights and present a coherent narrative. Additionally,

2025-02-18 19:40:02,235 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Goal Pass! Why do we need the PD (Placement Driver) microservice in TiDB, and in what scenarios or data volume should it be enabled?, evaluation result:{
  "accept": true,
  "plan_adjustment_suggestion": "The Final Answer effectively addresses the user's Goal by explaining the role of the PD component in TiDB and the scenarios where it is essential. However, the Plan could be improved by ensuring that the retrieval steps are more focused on the specific aspects of the Goal, such as the data volume thresholds for enabling PD. Additionally, the Plan could benefit from a more streamlined approach by reducing the number of retrieval steps and ensuring that each retrieval directly contributes to the final answer. Implementing dual retrieval for each query and immediately processing the results through the LLM generation tool could enhance efficiency and coherence.",
  "goal_classification": "Direct Problem Resolution"
}
